In [1]:
import torch
import numpy as np
import pandas as pd
from pathlib import Path

import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)


/Users/jliu/anaconda3/envs/neuron/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import sys
sys.path.append(str(Path.cwd()))

from src.neuron_analyzer.analysis.w_geometry import WeightGeometricAnalyzer


In [2]:
from transformers import AutoModelForCausalLM

model_name = 'EleutherAI/pythia-70m-deduped'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)
model.eval()


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): G

In [ ]:
# 构造 boost_neurons 和 suppress_neurons 的索引列表
# 你可以用自己的神经元索引，也可以随机选取（这里只做演示）
layer_num = 0  # 分析第0层
layer = dict(model.named_modules())[f'gpt_neox.layers.{layer_num}.mlp.dense_h_to_4h']
total_neurons = layer.weight.shape[0]

np.random.seed(42)
boost_neurons = np.random.choice(total_neurons, size=5, replace=False).tolist()
suppress_neurons = np.random.choice(list(set(range(total_neurons)) - set(boost_neurons)), size=5, replace=False).tolist()

print('Boost neurons:', boost_neurons)
print('Suppress neurons:', suppress_neurons)


In [ ]:
# 初始化分析器并运行分析
analyzer = WeightGeometricAnalyzer(
    model=model,
    layer_num=layer_num,
    boost_neurons=boost_neurons,
    suppress_neurons=suppress_neurons,
    device=device
)

# 运行分析
subspace_df, orthogonality_df = analyzer.run_analyses()

# 展示部分结果
display(subspace_df.head())
display(orthogonality_df.head())


In [ ]:
# 保存结果到CSV
subspace_df.to_csv('subspace_test.csv', index=False)
orthogonality_df.to_csv('orthogonality_test.csv', index=False)

# 简单可视化
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))
plt.hist(subspace_df['effective_dim'], bins=10)
plt.title('Effective Dimensionality Distribution')
plt.xlabel('Effective Dimensionality')
plt.ylabel('Count')
plt.show()
